In [ ]:
import requests

url = 'https://raw.githubusercontent.com/EinarOlafsson/spacr/main/spacr_classification.py'
response = requests.get(url)
with open('spacr_classification.py', 'w') as file:
    file.write(response.text)
    
import spacr_classification as spacr

In [ ]:
src = 'path'
spacr.generate_dataset(db_path=src+'/measurements/measurements.db',
                       dst='',
                       file_type='cell_png',
                       experiment='exp', 
                       sample=None)

In [ ]:
settings = {'train':True,
            'test': False,
            'classes':['nc','pc'],
            'model_type':'maxvit_t',
            'optimizer_type':'adamw',
            'schedule':'reduce_lr_on_plateau', #reduce_lr_on_plateau, step_lr
            'loss_type':'focal_loss', #binary_cross_entropy_with_logits, #focal_loss
            'normalize':True,
            'image_size':224,
            'batch_size':64,
            'epochs':100,
            'val_split':0.1,
            'train_mode':'erm',
            'learning_rate':0.0001,
            'weight_decay':0.00001,
            'dropout_rate':0.1,
            'init_weights':True,
            'amsgrad':True,
            'use_checkpoint':True,
            'gradient_accumulation':True,
            'gradient_accumulation_steps':4,
            'intermedeate_save':True,
            'pin_memory':True,
            'num_workers':30,
            'verbose':True}

model_path = 'path', #if using a custom trained model
model = torch.load(model_path) #if using a custom trained model
src = 'path'

spacr.train_test_model(src, settings, model=None)

In [ ]:
result_df = spacr.apply_model_to_tar(tar_path='path', 
                                     model_path='path', #this model is trained on nc vs pc, cells filtered on cell size, parasite size and recruitment >2 for nc and <2 for pc
                                     file_type='cell_png',
                                     image_size=224,
                                     batch_size=64,
                                     normalize=True,
                                     preload='images',
                                     num_workers=30,
                                     verbose=True)

In [ ]:
res_df = spacr.merge_pred_mes(src='path',
                        pred_loc='path',
                        target='protein', 
                        nucleus_dim=0, 
                        parasite_dim=2,
                        cell_dim=3, 
                        channel_of_interest=1,
                        parasite_size_min=0, 
                        nucleus_size_min=0,
                        cell_size_min=0,
                        parasite_min=0,
                        nucleus_min=0, 
                        cell_min=0,
                        target_min=0,
                        mask_chans=[0,1,2],
                        filter_data=False,
                        include_noninfected=False,
                        include_multiinfected=True, #bool or int
                        include_multinucleated=True,
                        cells_per_well=0,
                        save_filtered_filelist=False,
                        verbose=True)

In [ ]:
src = 'path'
dv_loc = src+'/measurements/'
spacr.generate_dependent_variable(df=res_df, 
                                  dv_loc=dv_loc, 
                                  pc_min=0.99, 
                                  nc_max=0.95, 
                                  agg_type='mean')

In [ ]:
coeffs = spacr.analyze_data_reg(sequencing_loc='path', 
                               dv_loc='path', 
                               dv_col = 'recruitment',
                               agg_type = 'mean',
                               transform='log',
                               min_cell_count=100, 
                               min_reads=75,
                               min_wells=1, 
                               max_wells=50,
                               min_frequency=0.05,
                               remove_outlier_genes=False, 
                               refine_model=True,
                               by_plate=False, 
                               regression_type='lasso',
                               alpha_value=0.000001,
                               fishers=True,
                               fisher_threshold=1.3)